# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [8]:
#because we only need to use the columns race and call, we will create a new dataframe and drop all the other info
df = data[['race','call']]

In [9]:
df.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [ ]:
1. What test is appropriate for this problem? Does CLT apply?

We want to compare 2 sample proportions ( not means) with known variances so the best test is a two sided z-test.
The data is independent and identically distributed, as defined in the problem statement.
CLT does not apply because there are more than 30 data points.

2. What are the null and alternate hypotheses?
The null hypothesis is that the difference between the rate of callbacks depending on race is zero. 
The alternate hypothesis is that the difference between the rate of callbacks depending on race is not zero.

In [7]:
3. Compute margin of error, confidence interval, and p-value
* Margin of Error                | 0.01528163
* 95% Confidence Interval        | 0.01675122 to 0.04731449
* z-test p-value                 | 0.00003984

In [10]:
df.groupby(['race','call']).size().apply(lambda x: x/2435.).unstack()

call,0.0,1.0
race,,
b,0.935524,0.064476
w,0.903491,0.096509


In [11]:
#number of callbacks for each race
num_call_w = sum(df[df.race=='w'].call)
num_call_b = sum(df[df.race=='b'].call)

#total rows for each race
num_tot_w = len(df[df.race=='w'].call)
num_tot_b = len(df[df.race=='b'].call)

#proportion of callback for each race
prop_w = num_call_w / num_tot_w
prop_b = num_call_b / num_tot_b

#pooled proportion
prop_pool = (prop_w * num_tot_w + prop_b * num_tot_b) / (num_tot_w + num_tot_b)

#difference in proportions
prop_diff = np.abs(prop_w-prop_b)

#standard error between proportions
std_error = np.sqrt(prop_pool * (1-prop_pool) * ((1./num_tot_w) + (1./num_tot_b)))

#z-score for this exercise
z = prop_diff / std_error

#Assume a confidence level of 0.95
CL = 0.95

#margin of error
margin_error = stats.norm.ppf(1-((1-CL)/2)) * std_error

#95% CI from the difference of proportions
ci_lower, ci_upper = stats.norm.interval(CL, loc=prop_diff, scale=std_error)

#p-value 
p_value = (1-stats.norm.cdf(z))*2

print("  %-30s | %s" % ('Calculation', 'Value'))
print(60 * '=')
print("  %-30s | %.8f" % ('Callback proportion for W', prop_w))
print("  %-30s | %.8f" % ('Callback proportion for B', prop_b))
print("  %-30s | %.8f" % ('Difference in proportions', prop_diff))
print("  %-30s | %.8f" % ('Pooled proportions', prop_pool))
print("  %-30s | %.8f" % ('Standard Error', std_error))
print("  %-30s | %.8f" % ('z-test statistic', z))
print("* %-30s | %.8f" % ('Margin of Error', margin_error))
print("* %-30s | %.8f to %.8f" % ('%i%% Confidence Interval' % (CL*100), ci_lower, ci_upper))
print("* %-30s | %.8f" % ('z-test p-value', p_value))

  Calculation                    | Value
  Callback proportion for W      | 0.09650924
  Callback proportion for B      | 0.06447639
  Difference in proportions      | 0.03203285
  Pooled proportions             | 0.08049281
  Standard Error                 | 0.00779689
  z-test statistic               | 4.10841215
* Margin of Error                | 0.01528163
* 95% Confidence Interval        | 0.01675122 to 0.04731449
* z-test p-value                 | 0.00003984


In [ ]:
4. Write a story describing the statistical significance in the context or the original problem.
The p-value from the z-test is very small and the CI supports the p-value b/c it does not include zero.
This means that the difference between the two proportions is not zero and we can reject the null hypothesis. 
This analysis indicates thatthere is a bias in who gets a callback for an employment application.

5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? 
If not, how would you amend your analysis?

There are obviously other factors that should be taken into account for this problem. Other factors
like education, gender, background and age also have to do with callbacks, and this is merely a correlation but
not a causation because it doesn't weigh all relevant factors. To improve this problem, you could map
out better causal relationships by performing more tests on other relationships of factors.